In [ ]:
# 1. Roboflow에서 데이터셋 다운로드
# pip install ultralytics roboflow
# pip install --upgrade ultralytics

from roboflow import Roboflow

rf = Roboflow(api_key="LvEPFGKVRzVtlM4WxitY")
project = rf.workspace("special-problem-manual").project("lung-cancer-detection-model")
version = project.version(1)
dataset = version.download("yolov12") 
# pip install -U albumentations opencv-python pydicom numpy torch torchvision ultralytics

 폐암 진단용 영상(CT/X-ray) 대상 탐지를 위해 전처리·증강·학습 파이프라인을 일원화
핵심 개선점
- CT(X-ray) 자동 분기 전처리:
  * CT: DICOM/NPY/PNG 등 입력 허용, HU window → intensity normalization, 3D에서 2D slice 지원
  * X-ray: CLAHE + intensity normalize
- 임상 신뢰성 중심 증강 설계(Albumentations):
  * 구조 보존(Structure-preserving)을 기본 원칙으로 한 기하학/강도 기반 증강
  * YOLO 형식 bbox와 동기화
- 설정(하이퍼파라미터) 분리: dataclass Config
- 재현성 관리: seed 고정, deterministic 옵션
- 학습 엔진: Ultralytics YOLO(v8/v12 호환) 사용 옵션
  * 미설치 환경에서도 스크립트 자체는 import/수정 가능 (런타임에서만 필요)
- 로깅: 간단한 CSV/JSON 로깅 및 콘솔 Pretty print

In [ ]:
from __future__ import annotations
import warnings
warnings.filterwarnings("ignore")

import os
import cv2
import json
import math
import time
import random
import shutil
import string
import typing as T
from dataclasses import dataclass, asdict
from ultralytics import YOLO
import numpy as np
import pydicom  # CT DICOM용 모듈
import albumentations as A. # pip install -U albumentations

In [ ]:
from ultralytics import settings

# View all settings
print(settings)

# Return a specific setting
value = settings["runs_dir"]

In [ ]:
# 2. 사전학습 모델 & data 불러오기
from ultralytics import YOLO
model = YOLO("/home/alpaco/homework/LungCa detection /yolo12m.pt")  

In [ ]:
#yaml 파일
yaml_path = "/home/alpaco/homework/LungCa detection /Lung-Cancer-Detection-(Model)-1/data.yaml"

# train -> 
model.train(data=yaml_path, epochs=3, imgsz=640)

In [ ]:
project="/home/alpaco/homework/outputs",  # 절대경로
name="yolov12m_test3",                                      
device="cuda"

In [ ]:
from ultralytics import YOLO

model = YOLO("/home/alpaco/homework/LungCa detection /best.pt")

# 모델 구조 확인
model.info()

In [ ]:
# 3. 전처리
import cv2, numpy as np, pydicom

def hu_window(img, window=(-1200, 400)):
    low, high = window
    img = np.clip(img, low, high)
    return ((img - low) / (high - low)).astype(np.float32)

def apply_clahe(img, clip=2.0, tile=(8,8)):
    img = np.uint8(np.clip(img * 255, 0, 255))
    clahe = cv2.createCLAHE(clipLimit=clip, tileGridSize=tile)
    img = clahe.apply(img)
    return img.astype(np.float32) / 255.0

In [ ]:
# 4. 증강
import albumentations as A

lung_augment = A.Compose([
    A.HorizontalFlip(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.4),
    A.GaussNoise(var_limit=(5.0, 20.0), p=0.2),  
    A.RandomGamma(gamma_limit=(90, 110), p=0.3)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

In [ ]:
from ultralytics import YOLO
import os

# 공백이 있는 폴더는 언더바로 교체 권장
model_path = "/home/alpaco/homework/LungCa detection /best.pt"
yaml_path = "/home/alpaco/homework/LungCa detection /Lung-Cancer-Detection-(Model)-1/data.yaml"

# 5. 모델 불러오기
model = YOLO(model_path)

# Fine-tuning
results = model.train(
    data=yaml_path,              # 절대경로로 변경!
    epochs=10,
    imgsz=640,
    batch=16,
    lr0=0.001,
    optimizer="SGD",
    device="cuda",
    project="/home/alpaco/homework/LungCa detection ",
    name="yolov12m_finetune",
    seed=42,
    deterministic=True,
    verbose=True
)

In [ ]:
# 6. VALIDATION
results_val = model.val(
    data=yaml_path,
    split="val",   # validation set
    imgsz=640,
    batch=8,
    conf=0.25,
    iou=0.5,
    save_json=True,
    save_hybrid=True
)
print(" Validation metrics:", results_val.results_dict)

#  7. TEST 
results_test = model.predict(
    source="/home/alpaco/homework/LungCa detection /Lung-Cancer-Detection-(Model)-1/test/images",  
    imgsz=640,
    conf=0.3,
    save=True,        # 결과 이미지 저장
    save_txt=True,    # bbox 좌표 txt 저장
    project="/home/alpaco/homework/outputs",
    name="yolov12m_test_pred"
)
print(" Test inference done! ")

In [ ]:
# 8. SAHI
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction

detection_model = AutoDetectionModel.from_pretrained(
    model_type="ultralytics",  # 또는 "yolov8" 등 호환 모델 타입
    model_path="/home/alpaco/homework/LungCa detection /yolov12m_finetune2/weights/best.pt",
    confidence_threshold=0.3,
    device="cuda"
)

In [ ]:
result = get_sliced_prediction(
   image="/home/alpaco/homework/outputs/yolov12m_test_pred/A0046_2-029-dcm_jpeg.rf.2bc85c14bfea3698435415e11f16963d.jpg",
    detection_model=detection_model,
    slice_height=512,
    slice_width=512,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2
)

In [ ]:
result.export_visuals(export_dir="/home/alpaco/homework/outputs/sahi_results")

In [ ]:
#9. Result 
mport matplotlib.pyplot as plt
import os
from PIL import Image

save_dir = "/home/alpaco/homework/outputs/sahi_results"
sample_img = [f for f in os.listdir(save_dir) if f.endswith(".png")][0]
img = Image.open(os.path.join(save_dir, sample_img))

plt.figure(figsize=(8, 8))
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
results = model.val(data= yaml_path,  imgsz=640)
print("mAP50-95:", results.box.map)      # mean Average Precision @ IoU 0.5–0.95
print(" mAP50:", results.box.map50)        # mean Average Precision @ IoU 0.5
print(" Precision:", results.box.p)        # Precision per class
print(" Recall:", results.box.r)           # Recall per class

In [ ]:
import pandas as pd 
df = pd.DataFrame([metrics])
os.makedirs("/home/alpaco/homework/outputs/metrics_report", exist_ok=True)
csv_path = "/home/alpaco/homework/outputs/metrics_report/yolov12m_metrics.csv"
df.to_csv(csv_path, index=False)
print(f" Metrics saved to {csv_path}")


In [ ]:
# graph 

precision = np.mean(results.box.p) 
recall = np.mean(results.box.r)    

metrics = {
    "mAP50-95": results.box.map,
    "mAP50": results.box.map50,
    "Precision": precision,
    "Recall": recall
}

df = pd.DataFrame([metrics])
os.makedirs("/home/alpaco/homework/outputs/metrics_report", exist_ok=True)
csv_path = "/home/alpaco/homework/outputs/metrics_report/yolov12m_metrics.csv"
df.to_csv(csv_path, index=False)
print(f"Metrics saved to {csv_path}")

In [ ]:
plt.figure(figsize=(6,4))
plt.bar(list(metrics.keys()), list(metrics.values()),
        color=['#4B9CD3', '#77DD77', '#FFB347', '#FF6961'])
plt.title("YOLOv12m Fine-tuning Performance")
plt.ylabel("Score (0–1)")
plt.ylim(0, 1.0)

for i, (k, v) in enumerate(metrics.items()):
    plt.text(i, v + 0.02, f"{v:.3f}", ha='center', fontsize=10)

plt.tight_layout()
plot_path = "/home/alpaco/homework/outputs/metrics_report/yolov12m_performance.png"
plt.savefig(plot_path)
plt.close()
print(f"✅ Performance chart saved to {plot_path}")


print("\n YOLOv12m Fine-tune Evaluation Summary")
for k, v in metrics.items():
    print(f"{k:<12}: {v:.4f}")

항목| 내용 | 

모듈 |sahi.models.yolov8 → YOLOv12도 내부 구조가 동일하므로 그대로 사용

모델 지정 |model_type="yolov12" 명시 (명시하지 않아도 작동하지만 명시 추천)

입력 이미지 |X-ray 또는 CT slice (512×512 이상 권장)

슬라이싱 |영상이 클 경우(>1024px) 작은 결절을 놓치지 않기 위해 분할 탐지 수행

결과 출력 |results/ 폴더 내에 시각화된 결과(.jpg, .json) 저장


                            지표          |    의미                                 |  기준
                            mAP50    |  IOU 0.5에서 평균정확도    |  ≥ 0.8 ✅
                            Recall      | 실제 병변 중 탐지된 비율      | ≥ 0.85 ✅
                            Precision | 탐지된 것 중 진짜 병변 비율 | ≥ 0.8 ✅

                            지표                   |    의미                       |  기준
                            False Negative| 병변을 놓친 비율        | 낮을수록 좋음
                            SAHI                  | 작은 병변 탐지 향상  |고해상도 영상에 효과적


In [ ]:
import pandas as pd

metrics = {
    "mAP50": [results.box.map50],
    "mAP50-95": [results.box.map]
}
df = pd.DataFrame(metrics)
df.to_csv("/home/alpaco/homework/outputs/yolo_metrics_summary.csv", index=False)
